In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import json
from glob import glob
from tqdm import tqdm
from PIL import Image

In [2]:
!mkdir images
!mkdir labels

In [3]:
!mkdir val-img

In [4]:
images = glob('/kaggle/input/xview-dataset/train_images/train_images/*')
for img in tqdm(images):
    image = Image.open(img).convert('L')
    name = img.split('/')[-1].split('.')[0]
    image.save('images/'+ name +'.tiff')

100%|██████████| 846/846 [04:32<00:00,  3.11it/s]


In [5]:
images = glob('/kaggle/input/xview-dataset/val_images/val_images/*')
for img in tqdm(images):
    image = Image.open(img).convert('L')
    name = img.split('/')[-1].split('.')[0]
    image.save('val-img/'+ name +'.tiff')

100%|██████████| 281/281 [01:37<00:00,  2.90it/s]


In [6]:
f = open('/kaggle/input/xview-dataset/train_labels/xView_train.geojson')

# returns JSON object as
# a dictionary
data = json.load(f)

In [7]:
classes_dict = {
    'small_vehicle':[17,18,36],
    'medium_vehicle':[20,34,37],
    'large_vehicle':[21,23,25,26,27,28,32,60,62,63,64,65,66,25],
    'bus':[19],
    'double_trailer_truck':[],
    'container':[91,35,57],
    'heavy_equipment':[56,53,59,61,29],
    'pylon':[93],
    'small_aircraft':[12,15,11],
    'large_aircraft':[11,13],
    'small_vessel':[41,42,47],
    'medium_vessel':[44,50,40],
    'large_vessel':[45,49,51,52]
}

In [8]:
def normalize_coordinates(row_i, col_j, img):
    num_rows, num_cols = img.shape[:2]
    x = col_j/(num_cols - 1.)
    y = row_i/(num_rows - 1.)
    return x, y

In [9]:
classes_dict.keys()

dict_keys(['small_vehicle', 'medium_vehicle', 'large_vehicle', 'bus', 'double_trailer_truck', 'container', 'heavy_equipment', 'pylon', 'small_aircraft', 'large_aircraft', 'small_vessel', 'medium_vessel', 'large_vessel'])

In [10]:
def create_points(txt, num_rows, num_cols):
    x1,y1,x2,y2 = txt.split(',')
    a = int(x1)/(num_cols - 1.)
    b = int(y1)/(num_rows - 1.)
    c = int(x2)/(num_cols - 1.)
    d = int(y2)/(num_rows - 1.)
    return ' '.join([str(abs((a+c)/2)),str(abs((b+d)/2)),str(abs((a - c))),str(abs((b - d)))])

In [11]:
create_points("10,25,36,25")

TypeError: create_points() missing 2 required positional arguments: 'num_rows' and 'num_cols'

In [11]:
def get_class(num):
    for k in classes_dict.keys():
        if num in classes_dict[k]:
            return k
    return 'err'

In [12]:
classes_dict1 = {
    'small_vehicle':0,
    'medium_vehicle':1,
    'large_vehicle':2,
    'bus':3,
    'double_trailer_truck':4,
    'container':5,
    'heavy_equipment':6,
    'pylon':7,
    'small_aircraft':8,
    'large_aircraft':9,
    'small_vessel':10,
    'medium_vessel':11,
    'large_vessel':12
}

In [13]:
abs(-10)

10

In [14]:
from tqdm import tqdm
import cv2

for i in tqdm()

SyntaxError: invalid syntax (3177976144.py, line 4)

In [14]:
from tqdm import tqdm
import cv2
import os
jsons_1 = {}
imag = list()

for i in tqdm(data['features']):
    catagory_num = i['properties']['type_id']
    class_mafat = get_class(catagory_num)
    if class_mafat != 'err':
        points = i['properties']['bounds_imcoords']
        img_id = i['properties']['image_id']
        if img_id not in imag:
            if(os.path.exists('/kaggle/input/xview-dataset/train_images/train_images/' + img_id) == 0):
                continue
            else:
                imag.append(img_id)
                img = cv2.imread("/kaggle/input/xview-dataset/train_images/train_images/"+ img_id)
                num_rows, num_cols = img.shape[:2]
        new_json = {
            'pixels':create_points(points, num_rows, num_cols),
            'type': classes_dict1.get(class_mafat)
        }
        if img_id not in jsons_1.keys():
            jsons_1[img_id] = []
        jsons_1[img_id].append(new_json)

100%|██████████| 601937/601937 [03:28<00:00, 2884.75it/s] 


In [15]:
print(new_json)

{'pixels': '0.6219088098918084 0.6673076923076924 0.007341576506955216 0.004615384615384577', 'type': 0}


In [16]:
for i in tqdm(jsons_1.keys()):
    path = 'labels/' + i.split('.')[0] + '.txt'
    f = open(path, "w")
    for k in jsons_1[i]:
        f.write(str(k['type']) + ' ' + k['pixels'] +'\n')
    f.close()

100%|██████████| 792/792 [00:00<00:00, 2904.23it/s]


In [27]:
!ls

LICENSE.md   data3.yaml  hubconf.py	   runs      train_aux.py
README.md    deploy	 inference	   scripts   utils
__pycache__  detect.py	 models		   test.py   wandb
cfg	     export.py	 paper		   tools     yolov7-tiny.pt
data	     figure	 requirements.txt  train.py  yolov7.pt


In [29]:
%cd ..

/kaggle/working


In [30]:
!ls

__notebook_source__.ipynb  labels	 val-img	yolov7
images			   labels.cache  val-img.cache


In [31]:
%cd labels

/kaggle/working/labels


In [32]:
!ls

10.txt	  1244.txt  1565.txt  1896.txt	223.txt   2521.txt  423.txt  682.txt
100.txt   1245.txt  1568.txt  1897.txt	2230.txt  2523.txt  43.txt   69.txt
102.txt   1246.txt  157.txt   1898.txt	2237.txt  2524.txt  430.txt  708.txt
1036.txt  125.txt   1577.txt  1899.txt	2239.txt  2530.txt  432.txt  709.txt
1037.txt  1253.txt  1579.txt  1900.txt	2242.txt  2531.txt  433.txt  711.txt
104.txt   1255.txt  158.txt   1902.txt	2250.txt  2532.txt  434.txt  712.txt
1042.txt  1256.txt  1580.txt  1904.txt	2251.txt  2538.txt  437.txt  716.txt
1044.txt  126.txt   1584.txt  1906.txt	2261.txt  254.txt   445.txt  72.txt
1046.txt  1265.txt  1585.txt  1907.txt	2264.txt  2541.txt  451.txt  720.txt
1048.txt  1268.txt  1586.txt  1908.txt	2268.txt  2542.txt  455.txt  724.txt
1049.txt  1278.txt  1587.txt  1910.txt	2269.txt  2543.txt  457.txt  727.txt
105.txt   128.txt   159.txt   1912.txt	2270.txt  2544.txt  46.txt   73.txt
1050.txt  1280.txt  1590.txt  1913.txt	2278.txt  2545.txt  462.txt  732.txt
1051.txt  1284.t

In [34]:
!cat 1036.txt

2 0.8627113337507827 -0.001088985687616677 0.00720100187852224 0.010267579340385812
2 0.27019411396368187 0.0006222775357809583 0.005009392611145913 0.005600497822028625
2 0.8148090169067 -0.001866832607342875 0.00720100187852224 0.008711885500933415
10 0.02864746399499061 0.6381456129433727 0.00469630557294928 0.004978220286247725
10 0.03052598622417032 0.6375233354075918 0.0040701314965560405 0.005600497822028649
10 0.03271759549154665 0.6361232109520847 0.004070131496556044 0.0059116365899191115
2 0.17031934877896054 0.30911636589919106 0.005635566687539145 0.0059116365899191115
10 0.0854727614276769 0.6294337274424393 0.004383218534752667 0.002489110143123807
10 0.08954289292423294 0.6288114499066584 0.004383218534752653 0.0031113876789048422
11 0.07201001878522229 0.5966085874299938 0.005009392611145899 0.004667081518357152
11 0.07138384470882905 0.5917859365276914 0.003757044458359421 0.0037336652146856553
0 0.03804007514088917 0.582451773490977 0.004696305572949283 0.00373366521

In [ ]:
!cat 2355.txt

In [26]:
Lines = open(glob('labels/*')[0]).readlines() 

In [27]:
name = list()
for i in jsons_1:
    name.append(i)

In [28]:
len(name)

792

In [29]:
len(jsons_1)

792

In [ ]:
import tensorflow.keras.preprocessing
import numpy as np
import os.path
train_image = []
for i in tqdm(range(len(name))):
    if(os.path.exists('/kaggle/input/xview-dataset/train_images/train_images/'+name[i]) == 0):
        continue
    else:
        img = tensorflow.keras.preprocessing.image.load_img('/kaggle/input/xview-dataset/train_images/train_images/'+name[i],target_size=(500,500,3))
        img = tensorflow.keras.preprocessing.image.img_to_array(img)
        img = img/255
        train_image.append(img)
X = np.array(train_image)

In [ ]:
len(X)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[0])

In [ ]:
X[0].shape

In [ ]:
list(jsons_1.items())[0]

In [ ]:
import cv2
 
# read an image
img = cv2.imread("/kaggle/input/xview-dataset/train_images/train_images/2400.tif")
 
# write it in a new format
img.shape

In [ ]:
y = np.array(jsons_1.values())

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7 # Downloading YOLOv7 repository and installing requirements
%cd yolov7
!pip3 install -qr requirements.txt
!pip3 install -q roboflow

Cloning into 'yolov7'...
remote: Enumerating objects: 1130, done.
remote: Total 1130 (delta 0), reused 0 (delta 0), pack-reused 1130
Receiving objects: 100% (1130/1130), 69.94 MiB | 16.65 MiB/s, done.
Resolving deltas: 100% (520/520), done.
/kaggle/working/yolov7


In [ ]:
!wget "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt"

In [ ]:
import os
import sys
import glob
import wandb
import torch
from roboflow import Roboflow
from kaggle_secrets import UserSecretsClient
from IPython.display import Image, clear_output, display  # to display images



print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
wandb.login(key = 'ae2d6048dc1dfdc6016150805e89224ab18ea529')

In [ ]:
wandb.init(project="yolov7-tiny",name=f"run13")

In [ ]:
rf = Roboflow(api_key='m9lVPO9GCk9rjZxf0TWE')

In [ ]:
workspace = rf.workspace()

In [ ]:
project = workspace.project("object-detection-in-satrlight-images")

In [ ]:
project.versions()

In [ ]:
for i in tqdm(range(len(name))):
    if(os.path.exists('/kaggle/input/xview-dataset/train_images/train_images/'+name[i]) == 0):
        continue
    else:
        project.upload("/kaggle/input/xview-dataset/train_images/train_images/"+name[i])

In [ ]:
dataset = project.version(2).download("yolov7")

In [ ]:
import json 
import yaml

In [ ]:
stream = open('hyp.scratch.p5.yaml', 'w')

In [ ]:
data = """lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.2  # image translation (+/- fraction)
scale: 0.9  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.15  # image mixup (probability)
copy_paste: 0.0  # image copy paste (probability)
paste_in: 0.15  # image copy paste (probability), use 0 for faster training
loss_ota: 1 # use ComputeLossOTA, use 0 for faster training"""

In [ ]:
yaml.dump(data, stream)

In [ ]:
stream1 = yaml.dump(data)

In [ ]:
import os.path

save_path = '/kaggle/working/'
completeName = os.path.join(save_path, "hyp.scratch.p5.yaml")         

file1 = open(completeName, "w")
file1.write(stream1)

file1.close()

In [ ]:
!python train.py --batch 64 --cfg cfg/training/yolov7-tiny.yaml --epochs 40 --data /kaggle/input/xview-data/data1.yaml --weights 'yolov7/yolov7-tiny.pt' --device 0 --entity 'yolov7-tiny' --project 'yolov7-tiny' --name 'run13'

In [ ]:
!ls

In [26]:
!wandb disabled
!python train.py --img 500 --batch 16 --epochs 50 --data data3.yaml  --weights 'yolov7.pt'

W&B disabled.
100%|██████████████████████████████████████| 72.1M/72.1M [00:03<00:00, 20.7MB/s]

/opt/conda/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Scanning images:   0%|                                  | 0/846 [00:00<?, ?it/s]train: WARNING: Ignoring corrupted image and/or label /kaggle/working/images/100.tiff: negative labels
train: WARNING: Ignoring corrupted image and/or label /kaggle/working/images/1036.tiff: negative labels
train: WARNING: Ignoring corrupted image and/or label /kaggle/working/images/1037.tiff: negative labels
train: WARNING: Ignoring corrupted image and/or label /kaggle/working/images/1044.tiff: non-normalized or out of bounds coordinate labels
train: WARNING: Ignoring corrupted image and

In [24]:
!pip install torch --upgrade

     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.8/887.5 MB 87.5 MB/s eta 0:00:08^C
     ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/887.5 MB 78.3 MB/s eta 0:00:09
ERROR: Operation cancelled by user


In [ ]:
data_yaml = dict(
    train = '/kaggle/working/images',
    val = '../val-img',
    nc = 13,
    names= ['small_vehicle','medium_vehicle','large_vehicle','bus','double_trailer_truck','container','heavy_equipment','pylon', 'small_aircraft','large_aircraft','small_vessel','medium_vessel','large_vessel']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('data3.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)